# GOME-2 functions

In [ ]:
def GOME_L3_version(component_nom, year, month):

    """ Get version of L3 GOME-2 dataset for each component nomenclature

         Args:
            component_nom (str): Component chemical nomenclature
            year (str): Year of dataset
            month (str): Month of dataset

        Returns:
            version (str): GOME-2 dataset version
    """
    
    year = int(year)
    month = int(month)
    
    if component_nom == 'NO2':
        version = 'v1'

    return version

In [ ]:
def GOME_L3_download(component_nom, date, satellite):

    """ Download L3 GOME-2 datasets

        Args:
            component_nom (str): Component chemical nomenclature
            date (str): Query date
            satellites (list): List with A, B and/or C referring to METOP series

        Returns:
            product_name (str): Product name of GOME-2 product
    """
    
    year = date.split('-')[0]
    month = date.split('-')[1]
    version = GOME_L3_version(component_nom, year, month)

    output_path = 'data/gome/' + component_nom + '/monthly/' + year + '-' + month
    os.makedirs(output_path, exist_ok = True) 

    product_name = ''.join(['GOME_', component_nom, '_Global_', year, month, '_METOP' + satellite + '_DLR_', version, '.nc'])
    path = 'ftp://acsaf.eoc.dlr.de/gome2' + satellite.lower() + '/level3/' + component_nom + '/' + year + '/' + product_name
   
    file_name = output_path + '/' + product_name
    subprocess.run(['wget', '-q', '-nc', path, '-O', file_name])

    if os.stat(file_name).st_size == 0:  
        os.remove(file_name) 
        print(product_name, 'is not available.')
        
    else:
        print(product_name, 'was downloaded.')

In [ ]:
def GOME_L3_read(component_nom, sensor_column, dates, lat_res = 0.25, lon_res = 0.25):

    """ Read L3 GOME-2 datasets as xarray dataset object and assign time

         Args:
            component_nom (str): Component chemical nomenclature
            sensor_column (str): Name of sensor column in downloaded dataset
            dates (list): Available dates
            lat_res (float): Spatial resolution for latitude
            lon_res (float): Spatial resolution for longitude

        Returns:
            sensor_ds (xarray): GOME-2 dataset in xarray format
    """

    if lat_res < 0.25 or lon_res < 0.25:
        print('To show the original data, the resolution must equal to 1x1º.')
        print('To show aggregated data, the resolution must be superior to 1x1º.')
        raise KeyboardInterrupt()

    sensor_ds_all = []

    for date in dates:

        year = date.split('-')[0]
        month = date.split('-')[1]

        sensor_ds_ABC = []

        # Combine data from METOP-A, METOP-B and METOP-C
        path = 'data/gome/' + component_nom + '/monthly/' + year + '-' + month
        product_names = [file for file in os.listdir(path)]
        
        for product_name in product_names:
            
            sensor_ds_sat = xr.open_dataset(path + '/' + product_name)
            sensor_ds_int_sat = xr.open_dataset(path + '/' + product_name, group = 'PRODUCT')
            sensor_ds_sat[sensor_column] = sensor_ds_int_sat[sensor_column]
            unit = sensor_ds_sat[sensor_column].units
            sensor_ds_ABC.append(sensor_ds_sat)

        sensor_ds_ABC = xr.concat(sensor_ds_ABC, dim = 'latitude')

        # Regrid onto a custom defined regular grid
        lat_bins = np.arange(-90, 90 + lat_res/2, lat_res)
        lon_bins = np.arange(-180, 180 + lon_res/2, lon_res)
        
        lat_center = np.arange(-90 + lat_res/2, 90, lat_res)
        lon_center = np.arange(-180 + lon_res/2, 180, lon_res)

        sensor_ds_ABC_gridded_int = sensor_ds_ABC.groupby_bins('latitude', lat_bins, labels = lat_center).mean()
        sensor_ds_ABC_gridded = sensor_ds_ABC_gridded_int.groupby_bins('longitude', lon_bins, labels = lon_center).mean()
        sensor_ds_ABC_gridded = sensor_ds_ABC_gridded.rename({'latitude_bins': 'latitude', 'longitude_bins': 'longitude'})

        # Add time
        time_str = dt.datetime(int(year), int(month), 1)
        sensor_ds_ABC_gridded = sensor_ds_ABC_gridded.assign_coords({'time': time_str}).expand_dims(dim = ['time'])

        # Add units as attribute
        sensor_ds_ABC_gridded.attrs['units'] = unit

        sensor_ds_all.append(sensor_ds_ABC_gridded)
        
    sensor_ds = xr.concat(sensor_ds_all, dim = 'time')
    sensor_ds = sensor_ds.rename({sensor_column: 'sensor_column'})
    
    return sensor_ds

In [ ]:
def GOME_L2_download(component_nom, date, satellite):

    """ Download L2 GOME-2 datasets

         Args:
            component_nom (str): Component chemical nomenclature
            date (str): Query date
            satellites (list): List with A, B and/or C referring to METOP series
    """
    
    output_path = 'data/gome/' + component_nom + '/' + date + '/' + satellite
    os.makedirs(output_path, exist_ok = True) 

    # Get year, month and day from date
    year = date.split('-')[0]
    month = date.split('-')[1]
    day = date.split('-')[2]

    # Save index.html with available offline products through FTP for specific date
    date_path = 'ftp://acsaf.eoc.dlr.de/gome2' + satellite.lower() + '/offline/' + year + '/' + month + '/' + day + '/'
    date_url = 'data/gome/' + component_nom + '/' + 'index.html'
    subprocess.run(['wget', '-q', '-O', date_url, '-i', date_path])

    # Download all files for date and satellite
    if os.stat('data/gome/' + component_nom + '/' + 'index.html').st_size != 0:  
            
        # Read index.html and get content within pre tabs
        html_text = open('data/gome/' + component_nom + '/' + 'index.html', 'r')
        items_int = bs4.BeautifulSoup(html_text, 'lxml').pre.get_text().splitlines()[1:]
        items = [item_int.split('File        ', 1)[1].split('  (', 1)[0] for item_int in items_int]

        for product_name in items:
            
            file_name = output_path + '/' + product_name
            path = 'ftp://acsaf.eoc.dlr.de/gome2' + satellite.lower() + '/offline/' + year + '/' + month + '/' + day + '/' + product_name
            subprocess.run(['wget', '-q', '-nc', path, '-O', file_name])

            if os.stat(file_name).st_size == 0:  
                os.remove(file_name) 
                print(product_name, 'is not available.')
                
            else:
                print(product_name, 'was downloaded.')
    else:
        print(f'The datasets for {date} and METOP-{satellite} are not available.')

In [ ]:
def GOME_L2_read(component_nom, sensor_column, dates, satellites, lat_res = 0.25, lon_res = 0.25):

    """ Read L2 GOME-2 datasets as xarray dataset object and assign time

         Args:
            component_nom (str): Component chemical nomenclature
            sensor_column (str): Name of sensor column in downloaded dataset
            dates (list): Available dates
            satellites (list): List with A, B and/or C referring to METOP series
            lat_res (float): Spatial resolution for latitude
            lon_res (float): Spatial resolution for longitude
            
        Returns:
            sensor_ds (xarray): GOME-2 dataset in xarray format
    """

    if lat_res < 0.25 or lon_res < 0.25:
        print('To show the original data, the resolution must equal to 1x1º.')
        print('To show aggregated data, the resolution must be superior to 1x1º.')
        raise KeyboardInterrupt()

    sensor_ds_all = []

    # Concatenate all the products for different dates
    for date in dates:

        year = date.split('-')[0]
        month = date.split('-')[1]
        day = date.split('-')[2]
        
        sensor_ds_ABC = []

        # Concatenate all the products for METOP-A, B and C
        for satellite in os.listdir('data/gome/' + component_nom + '/' + date):

            sensor_ds_sat_daily = []
            path = 'data/gome/' + component_nom + '/' + date + '/' + satellite
            product_names = [file for file in os.listdir(path)]
            
            # Concatenate all the products for different delta times
            for product_name in product_names:

                f = h5py.File(path + '/' + product_name, 'r')
                f['TOTAL_COLUMNS/' + component_nom]

                latitude = f['GEOLOCATION/LatitudeCentre']
                longitude = f['GEOLOCATION/LongitudeCentre']

                # Get component for one delta time 
                sensor_ds_sat_time = xr.DataArray(
                                                f['TOTAL_COLUMNS/' + component_nom],
                                                dims = ('ground_pixel'),
                                                coords = {
                                                    'latitude': ('ground_pixel', latitude[:]),
                                                    'longitude': ('ground_pixel', longitude[:])
                                                },
                                                name = component_nom
                )
                
                # Get delta time from product name and add it as a variable
                delta_time_str_int = product_name.split(date.replace('-', ''), 1)[1].split('_', 1)[0]
                delta_time_str = [delta_time_str_int[i:i+2] for i in range(0, len(delta_time_str_int), 2)]
                delta_time_hour = delta_time_str[0]
                delta_time_minute = delta_time_str[1]
                delta_time_second = delta_time_str[2]
            
                delta_time = dt.datetime(int(year), int(month), int(day), 
                                                                int(delta_time_hour), int(delta_time_minute), 
                                                                int(delta_time_second))

                sensor_ds_delta_time = xr.DataArray(np.repeat(delta_time, len(sensor_ds_sat_time.ground_pixel)),
                                                    dims = ('ground_pixel'),
                                                    coords = {
                                                        'latitude': ('ground_pixel', latitude[:]),
                                                        'longitude': ('ground_pixel', longitude[:])
                                                    },
                                                    name = 'delta_time'
                )

                # Merge both variables (component and delta time)
                sensor_ds_sat_time = xr.merge([sensor_ds_sat_time, sensor_ds_delta_time])
                sensor_ds_sat_daily.append(sensor_ds_sat_time)
            
            sensor_ds_sat_daily = xr.concat(sensor_ds_sat_daily, dim = 'ground_pixel')
            
        sensor_ds_ABC = xr.concat([sensor_ds_sat_daily], dim = 'ground_pixel')

        sensor_ds_ABC = sensor_ds_ABC.assign_coords(longitude = (((sensor_ds_ABC.longitude + 180) % 360) - 180))

        y = sensor_ds_ABC.latitude.data
        x = sensor_ds_ABC.longitude.data
        z = sensor_ds_ABC[component_nom].data

        zi, yi, xi = np.histogram2d(y, x, bins = (180, 360), weights = z, normed = False)
        counts, _, _ = np.histogram2d(y, x, bins = (180, 360))
        zi = zi / counts
        
        sensor_ds_ABC_gridded = xr.DataArray(
                                            zi,
                                            dims = ['latitude', 'longitude'],
                                            coords = {
                                                'latitude': (['latitude'], yi[:-1]),
                                                'longitude': (['longitude'], xi[:-1])
                                            },
                                            name = 'sensor_column'
        )

        # Regrid onto a custom defined regular grid
        lat_bins = np.arange(-90, 90 + lat_res/2, lat_res)
        lon_bins = np.arange(-180, 180 + lon_res/2, lon_res)
        
        lat_center = np.arange(-90 + lat_res/2, 90, lat_res)
        lon_center = np.arange(-180 + lon_res/2, 180, lon_res)

        sensor_ds_ABC_gridded_int = sensor_ds_ABC_gridded.groupby_bins('latitude', lat_bins, labels = lat_center).mean()
        sensor_ds_ABC_gridded = sensor_ds_ABC_gridded_int.groupby_bins('longitude', lon_bins, labels = lon_center).mean()
        sensor_ds_ABC_gridded = sensor_ds_ABC_gridded.rename({'latitude_bins': 'latitude', 'longitude_bins': 'longitude'})

        # Get datafame with delta time (non-gridded)
        delta_time_df = pd.DataFrame()
        delta_time_df['ground_pixel'] = sensor_ds_ABC.ground_pixel
        delta_time_df['latitude'] = delta_time_df.apply(lambda row: sensor_ds_ABC.sel(ground_pixel = row['ground_pixel']).latitude.values, axis = 1)
        delta_time_df['longitude'] = delta_time_df.apply(lambda row: sensor_ds_ABC.sel(ground_pixel = row['ground_pixel']).longitude.values, axis = 1)
        delta_time_df['delta_time'] = delta_time_df.apply(lambda row: sensor_ds_ABC.sel(ground_pixel = row['ground_pixel']).delta_time.values, axis = 1)
        
        # Round latitude and longitude to 0 decimals
        delta_time_df_rounded = delta_time_df
        delta_time_df_rounded['latitude_rounded'] = [round(value.item(0)) for value in delta_time_df_rounded['latitude'].values]
        delta_time_df_rounded['longitude_rounded'] = [round(value.item(0)) for value in delta_time_df_rounded['longitude'].values]
        delta_time_df_rounded

        # Get datafame without delta time (gridded)
        delta_time_df_gridded = sensor_ds_ABC_gridded.to_dataframe()
        delta_time_df_gridded = delta_time_df_gridded.reset_index()

        # Round latitude and longitude to 0 decimals
        delta_time_df_gridded_rounded = delta_time_df_gridded
        delta_time_df_gridded_rounded['latitude_rounded'] = round(delta_time_df_gridded_rounded['latitude'])
        delta_time_df_gridded_rounded['longitude_rounded'] = round(delta_time_df_gridded_rounded['longitude'])

        # Order dataframes before merge
        delta_time_df_gridded_rounded = delta_time_df_gridded_rounded.sort_values(by = ['latitude_rounded', 'longitude_rounded'])
        delta_time_df_rounded = delta_time_df_rounded.drop(columns = 'ground_pixel').sort_values(by = ['latitude_rounded', 'longitude_rounded'])
        
        # Merge and clean
        final = pd.merge(delta_time_df_gridded_rounded, delta_time_df_rounded, 
                         on= ['latitude_rounded', 'longitude_rounded'], how = 'left')
        final = final[~(final.delta_time.isnull() & final['sensor_column'].notnull())]
        final = final.drop(columns = ['latitude_rounded', 'longitude_rounded', 'latitude_y', 'longitude_y'])
        final = final.rename({'latitude_x': 'latitude', 'longitude_x': 'longitude'}, axis = 1)
        final = final.set_index(['latitude', 'longitude'])
        final = final[~final.index.duplicated()].to_xarray()
        
        # Assign day as time
        time_str = dt.datetime(int(year), int(month), int(day))
        final = final.assign_coords({'time': time_str}).expand_dims(dim = ['time'])

        sensor_ds_all.append(final)

    sensor_ds = xr.concat(sensor_ds_all, dim = 'time')
   
    return sensor_ds